# च्याट इतिहास घटाउने सुविधा र एजेन्ट स्क्र्याचप्याडको प्रयोग सेम्यान्टिक कर्नेलमा

यो नोटबुकले सेम्यान्टिक कर्नेलको च्याट इतिहास घटाउने सुविधा र एजेन्ट स्क्र्याचप्याडको प्रयोग कसरी गर्ने भनेर देखाउँछ। यो लामो कुराकानीहरूलाई टोकन सीमा नपार गर्दै प्रभावकारी एआई एजेन्ट निर्माण गर्न आवश्यक छ।

## तपाईंले के सिक्नुहुनेछ:
1. **च्याट इतिहास घटाउने सुविधा**: टोकनको प्रयोग व्यवस्थापन गर्न कुराकानीको इतिहासलाई स्वचालित रूपमा संक्षेप गर्ने तरिका
2. **एजेन्ट स्क्र्याचप्याड**: प्रयोगकर्ताको प्राथमिकता र पूरा गरिएका कार्यहरू ट्र्याक गर्नको लागि स्थायी मेमोरी प्रणाली
3. **टोकन प्रयोग ट्र्याकिङ**: इतिहास घटाउने सुविधा प्रयोग गर्दा र नगर्दा टोकनको प्रयोग कसरी परिवर्तन हुन्छ भन्ने निगरानी गर्ने

## आवश्यकताहरू:
- Azure OpenAI सेटअप र वातावरण चरहरू कन्फिगर गरिएको
- अघिल्लो पाठहरूबाट आधारभूत एजेन्ट अवधारणाहरूको समझ


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## एजेन्ट स्क्र्याचप्याड बुझ्दै

### एजेन्ट स्क्र्याचप्याड के हो?

**एजेन्ट स्क्र्याचप्याड** एउटा स्थायी मेमोरी प्रणाली हो जुन एजेन्टहरूले प्रयोग गर्छन्:
- **पूरा गरिएका कार्यहरू ट्र्याक गर्न**: प्रयोगकर्ताका लागि के काम गरियो भनेर रेकर्ड राख्न
- **प्रयोगकर्ताका प्राथमिकताहरू भण्डारण गर्न**: मनपर्ने, नमनपर्ने र आवश्यकताहरू सम्झन
- **सन्दर्भ कायम गर्न**: कुराकानीहरूमा महत्त्वपूर्ण जानकारी पहुँचयोग्य राख्न
- **पुनरावृत्ति घटाउन**: बारम्बार एउटै प्रश्न सोध्नबाट बच्न

### यो कसरी काम गर्छ:
1. **लेख्ने प्रक्रिया**: एजेन्टले नयाँ जानकारी सिकेपछि स्क्र्याचप्याड अपडेट गर्छ
2. **पढ्ने प्रक्रिया**: निर्णय लिँदा एजेन्टले स्क्र्याचप्याडलाई सल्लाह लिन्छ
3. **स्थायित्व**: जानकारी च्याट इतिहास घट्दा पनि जीवित रहन्छ

यसलाई एजेन्टको व्यक्तिगत नोटबुकको रूपमा सोच्न सकिन्छ, जसले कुराकानीको इतिहासलाई पूरक बनाउँछ।


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## एजेन्ट स्क्र्याचप्याड प्लगइन बनाउनुहोस्

यो प्लगइनले एजेन्टलाई स्थायी स्क्र्याचप्याड फाइलमा पढ्न र लेख्न अनुमति दिन्छ।


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## च्याट इतिहास घटाउने प्रक्रिया आरम्भ गर्नुहोस्

ChatHistorySummarizationReducer ले कुराकानीको इतिहासले तोकिएको सीमा नाघेपछि स्वचालित रूपमा त्यसलाई संक्षेपमा प्रस्तुत गर्दछ।


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## छुट्टी योजना एजेन्ट बनाउनुहोस्

यो एजेन्टले प्रयोगकर्ताहरूलाई छुट्टी योजना बनाउन मद्दत गर्नेछ र स्क्र्याचप्याड मार्फत सन्दर्भ कायम राख्नेछ।


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immediately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## छुट्टी योजना संवाद चलाउनुहोस्

अब हामी एउटा पूर्ण संवादको माध्यमबाट जान्छौं जसले प्रदर्शन गर्दछ:
1. प्रारम्भिक योजना अनुरोध
2. प्राथमिकता संकलन
3. यात्रा कार्यक्रम निर्माण
4. स्थान परिवर्तन
5. च्याट इतिहास घटाउने
6. स्क्र्याचप्याड प्रयोग


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## नतिजाहरूको विश्लेषण

हाम्रो कुराकानीको क्रममा के भयो भनेर विश्लेषण गरौं:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## मुख्य बुँदाहरू

### १. च्याट इतिहास घटाउने
- **स्वचालित ट्रिगर**: सन्देशहरूको संख्या थ्रेसहोल्ड नाघेपछि घटाउने प्रक्रिया सुरु हुन्छ
- **टोकन बचत**: संक्षेपिकरणपछि टोकन प्रयोगमा उल्लेखनीय कमी
- **सन्दर्भ संरक्षण**: महत्त्वपूर्ण जानकारी संक्षेपमा सुरक्षित रहन्छ

### २. एजेन्ट स्क्र्याचप्याडका फाइदाहरू
- **स्थायी स्मृति**: प्रयोगकर्ताका प्राथमिकताहरू इतिहास घटाएपछि पनि सुरक्षित रहन्छन्
- **कार्य ट्र्याकिङ**: एजेन्टले सम्पन्न कार्यहरूको रेकर्ड राख्छ
- **सुधारिएको अनुभव**: प्राथमिकताहरू दोहोर्याउन आवश्यक पर्दैन

### ३. टोकन प्रयोगका ढाँचाहरू
- **रेखीय वृद्धि**: प्रत्येक सन्देशसँगै टोकनको संख्या बढ्छ
- **नाटकीय गिरावट**: घटाउने प्रक्रियाले टोकनको संख्या उल्लेखनीय रूपमा घटाउँछ
- **दिगो संवाद**: सीमाभित्र लामो अन्तरक्रिया सम्भव बनाउँछ


## सफाइ

यस डेमोको क्रममा बनाइएको स्क्र्याचप्याड फाइल सफा गर्नुहोस्:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# सारांश

बधाई छ! तपाईंले उन्नत सन्दर्भ व्यवस्थापन क्षमतासहितको एआई एजेन्ट सफलतापूर्वक कार्यान्वयन गर्नुभएको छ:

## तपाईंले के सिक्नुभयो:
- **च्याट इतिहास घटाउने**: टोकन सीमाहरू व्यवस्थापन गर्न वार्तालापलाई स्वचालित रूपमा संक्षेप गर्नुहोस्
- **एजेन्ट स्क्र्याचप्याड**: प्रयोगकर्ता प्राथमिकता र पूरा गरिएका कार्यहरूको लागि स्थायी मेमोरी कार्यान्वयन गर्नुहोस्
- **टोकन व्यवस्थापन**: लामो वार्तालापमा टोकनको प्रयोग ट्र्याक र अनुकूलित गर्नुहोस्
- **सन्दर्भ संरक्षण**: वार्तालाप संक्षेपमा महत्त्वपूर्ण जानकारी कायम राख्नुहोस्

## वास्तविक-विश्व प्रयोगहरू:
- **ग्राहक सेवा बोटहरू**: सत्रहरूमा ग्राहक प्राथमिकता सम्झनुहोस्
- **व्यक्तिगत सहायकहरू**: चलिरहेका परियोजना र प्रयोगकर्ता बानीहरू ट्र्याक गर्नुहोस्
- **शैक्षिक ट्यूटरहरू**: विद्यार्थीको प्रगति र सिकाइ प्राथमिकता कायम राख्नुहोस्
- **स्वास्थ्य सेवा सहायकहरू**: टोकन सीमाहरूको सम्मान गर्दै बिरामीको इतिहास राख्नुहोस्

## आगामी कदमहरू:
- थप परिष्कृत स्क्र्याचप्याड स्किमाहरू कार्यान्वयन गर्नुहोस्
- बहु-प्रयोगकर्ता परिदृश्यहरूको लागि डाटाबेस भण्डारण थप्नुहोस्
- डोमेन-विशिष्ट आवश्यकताहरूको लागि अनुकूलन संक्षेप रणनीतिहरू सिर्जना गर्नुहोस्
- सेम्यान्टिक मेमोरी खोजको लागि भेक्टर डाटाबेससँग संयोजन गर्नुहोस्
- पूर्ण सन्दर्भसहित दिनहरू पछि वार्तालाप पुनः सुरु गर्न सक्ने एजेन्टहरू निर्माण गर्नुहोस्



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
